Copyright 2023-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Shear Kernel Tutorial

This notebook demonstrates how to use the specialized lensing shear kernel (hard-coded to RBF at the moment).

⚠️ _Note that this is still an experimental feature._ ⚠️

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from MuyGPyS._src.gp.tensors import _pairwise_differences
from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Isotropy, l2, F2
from MuyGPyS.gp.hyperparameter import Parameter
from MuyGPyS.gp.kernels.experimental import ShearKernel

This is required to import the implementation from Bob Armstrong's original repository.
It must be cloned in the same directory as MuyGPyS for the relative paths to work.

In [ ]:
import importlib.util
import sys
spec_analytic = importlib.util.spec_from_file_location("analytic_kernel", "../../shear_kernel/analytic_kernel.py")
bar = importlib.util.module_from_spec(spec_analytic)
sys.modules["analytic_kernel"] = bar
spec_analytic.loader.exec_module(bar)
from analytic_kernel import shear_kernel

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

Here we build some simple data, which is mean to represent a grid of sky coordinates.

In [ ]:
n = 25  # number of galaxies on a side
xmin = 0
xmax = 1
ymin = 0
ymax = 1

xx = np.linspace(xmin, xmax, n)
yy = np.linspace(ymin, ymax, n)

x, y = np.meshgrid(xx, yy)
features = np.vstack((x.flatten(), y.flatten())).T
diffs = _pairwise_differences(features)
length_scale = 0.5

Use an Isotropic distance functor.

In [ ]:
dist_fn = Isotropy(
    metric=F2,
    length_scale=Parameter(length_scale),
)

Here we construct a shear value kernel (partial differential components of RBF), as well as the original RBF kernel using Bob's implementation.

In [ ]:
def original_shear(features, length_scale=1.0):
    n, _ = features.shape
    vals = np.zeros((3 * (n), 3 * (n)))
    vals[:] = np.nan
    for i, (ix, iy) in enumerate(features):
        for j, (jx, jy) in enumerate(features):
            tmp = shear_kernel(ix, iy, jx, jy, b=length_scale)
            for a in range(3):
                for b in range(3):
                    vals[(a * n) + i, (b * n) + j] = tmp[a, b]
            
    return vals

In [ ]:
K_analytic = original_shear(features, length_scale=length_scale)

Here we do the same using the MuyGPyS implementation. Note the increased efficiency.

In [ ]:
K_muygps = ShearKernel(deformation=dist_fn)(diffs)

Do the two implementations agree?

In [ ]:
np.allclose(K_analytic, K_muygps)

Plot results of the baseline and internal implementations. 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(K_analytic)
axes[0].set_title("original shear kernel")
axes[1].imshow(K_muygps)
axes[1].set_title("MuyGPyS shear kernel")
plt.show()

Runtime comparison of the two implementations:

In [ ]:
if True:
    %timeit original_shear(features)
    %timeit ShearKernel(deformation=dist_fn)(diffs)